In [5]:
import librosa
from librosa.core.spectrum import amplitude_to_db
import scipy.special as sp
import scipy.integrate as inte
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

def Gan_mmse(ksi, gamma):
    c = np.sqrt(np.pi) / 2
    v = gamma * ksi / (1 + ksi)
    j_0 = sp.iv(0, v / 2)
    j_1 = sp.iv(1, v / 2)
    C = np.exp(-0.5 * v)
    A = ((c * (v ** 0.5)) * C) / gamma
    B = (1 + v) * j_0 + v * j_1
    hw = A * B
    return hw

def Gan_log_mmse(ksi,gamma):
    def integrand(t):
        return np.exp(-t) / t
    A = ksi / (1 + ksi) 
    v = A * gamma
    ei_v = np.zeros(len(v))
    for i in range(len(v)): 
        ei_v[i] = 0.5 * inte.quad(integrand,v[i],np.inf)[0]
    hw = A * np.exp(ei_v)
    return hw
    
def Gan_log_mmse2(ksi,gamma):
    A = ksi / (1 + ksi) 
    v = A * gamma
    ei_v = np.zeros(len(v))
    for i in range(len(v)): 
        
        if v[i]<0.1:
            ei_v[i] = -2.3*np.log10(v[i])-0.6
        elif v[i]>=0.1 and v[i]<1:
            ei_v[i] = -1.544*np.log10(v[i]) + 0.166
        else:
            ei_v[i] = np.power(10,-0.53*v[i]-0.26)
      
    hw = A * np.exp(0.5*ei_v)
    return hw

def Gan_sqr_mmse(ksi,gamma):
    A = ksi / (1 + ksi) 
    v = A * gamma
    
    B = (1 + v) / gamma
    hw = np.sqrt(A * B)
    
    return hw

def enh_mmse(noisy, noise, params):
    n_fft = params['n_fft']
    hop_length = params['hop_length']
    win_length = params['win_length']
    
    S_noisy = librosa.stft(noisy, n_fft = n_fft, hop_length = hop_length, win_length = win_length)
    S_noise = librosa.stft(noise, n_fft = n_fft, hop_length = hop_length, win_length = win_length)

    phase_noisy = np.angle(S_noisy)
    mag_noisy = np.abs(S_noisy)
    
    D, T = np.shape(mag_noisy)
    
    mag_noise = np.mean(np.abs(S_noise), axis = 1)
    power_noise = mag_noise ** 2
    
    mag_enhence = np.zeros([D, T])
    aa = params['a_DD']
    
    power_enhence_frame = mag_noisy[:, 0] ** 2
    for i in range(T):
        # 获取每一帧的能量谱和幅度谱
        mag_frame = mag_noisy[:, i]
        power_frame = mag_frame ** 2
        
        # 获取用来进行VAD计算的信噪比
        SNR_VAD = 10 * np.log10(np.sum(power_frame) / np.sum(power_noise))
        
        # 计算后验信噪比
        gamma = np.minimum(power_frame / power_noise, params['max_gamma'])
        
        if i == 0:
            ksi = aa + (1 - aa) * np.maximum(gamma - 1, 0)
        else:
            ksi = aa * power_enhence_frame / power_noise + (1 - aa) * np.maximum(gamma - 1, 0)
            ksi = np.maximum(params['ksi_min'], ksi)
            
        mu = params['mu_VAD']
        if SNR_VAD < params['th_VAD']:
            power_noise = mu * power_noise + (1 - mu) * power_frame
        
        H = params['fun_GAN'](ksi, gamma)
        
        mag_enhence_frame = H * mag_frame
        mag_enhence[:, i] = mag_enhence_frame
        
        power_enhence_frame = mag_enhence_frame ** 2
        
    S_enhence = mag_enhence * np.exp(1j * phase_noisy)
    
    enhence = librosa.istft(S_enhence, hop_length = hop_length, win_length = win_length)
    return enhence

if __name__ == '__main__':
    noisy_file_path = r'E:\z_one\test_wav_by_myself\z_one_test_wav_hy_myself.wav'
    noisy, fs = librosa.load(noisy_file_path, sr = None)
    noise = noisy[:8000]
    
    params_mmse = {
        'n_fft' : 256,
        'hop_length' : 128,
        'win_length' : 256,
        'max_gamma' : 40,
        'a_DD' : 0.98,
        'ksi_min' : 10 ** (-25 / 10),
        'mu_VAD' : 0.98,
        'th_VAD' : 3,
        'fun_GAN' : Gan_log_mmse2
    }
    
    enhence = enh_mmse(noisy, noise, params_mmse)
    sf.write(r'E:\z_one\test_wav_by_myself\z_one_test_wav_hy_myself_ns_mmse.wav', enhence, fs)